### Importation des bibliothèques pour explorer les données:

In [2]:

import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt

### Lecture du dataset dans la variable data

In [3]:
data = pd.read_csv("data/original.csv")

### Affichage du dataset

In [5]:
data

,clientid,income,age,loan,default
0,1,66155.925095,59.017015,8106.532131,0
1,2,34415.153966,48.117153,6564.745018,0
2,3,57317.170063,63.108049,8020.953296,0
3,4,42709.534201,45.751972,6103.642260,0
4,5,66952.688845,18.584336,8770.099235,1
...,...,...,...,...,...
1995,1996,59221.044874,48.518179,1926.729397,0
1996,1997,69516.127573,23.162104,3503.176156,0
1997,1998,44311.449262,28.017167,5522.786693,1
1998,1999,43756.056605,63.971796,1622.722598,0


### Quelques informations sur le dataset


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   clientid  2000 non-null   int64  
 1   income    2000 non-null   float64
 2   age       1997 non-null   float64
 3   loan      2000 non-null   float64
 4   default   2000 non-null   int64  
dtypes: float64(3), int64(2)
memory usage: 78.2 KB


### Suppression des lignes dont au moins une case est nulle

Nous observons que toutes les colonnes ne comportent pas de données manquantes sauf la colonne "age" qui contient 3 valeurs nullles. Nous allons supprimer les lignes qui contiennent ces valeurs nulles étant donné qu'elles ne représentent que 0,0015% de notre dataset.

La variable qui nous intéresse (en d'autres termes notre "target") est la variable default qui montre si l'individu est un bon client (default=0) ou pas (default=1).

### Suppression des lignes dont au moins une case est nulle et la suppression des colonnes : clientid et default

In [16]:
no_null_data = data[['income','age','loan']]
no_null_data

,income,age,loan
0,66155.925095,59.017015,8106.532131
1,34415.153966,48.117153,6564.745018
2,57317.170063,63.108049,8020.953296
3,42709.534201,45.751972,6103.642260
4,66952.688845,18.584336,8770.099235
...,...,...,...
1995,59221.044874,48.518179,1926.729397
1996,69516.127573,23.162104,3503.176156
1997,44311.449262,28.017167,5522.786693
1998,43756.056605,63.971796,1622.722598


### Vérification

In [10]:
no_null_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1997 entries, 0 to 1999
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   clientid  1997 non-null   int64  
 1   income    1997 non-null   float64
 2   age       1997 non-null   float64
 3   loan      1997 non-null   float64
 4   default   1997 non-null   int64  
dtypes: float64(3), int64(2)
memory usage: 93.6 KB


En effet, nous remarquons maintenant qu'il n'existe plus de cases vides.

### Description plus détaillée de la data (sans valeurs nulles)

In [11]:
no_null_data[]

,clientid,income,age,loan,default
count,1997.000000,1997.000000,1997.000000,1997.000000,1997.000000
mean,1001.956935,45333.864334,40.807559,4445.487716,0.141713
std,576.702206,14325.131177,13.624469,3046.792457,0.348842
min,1.000000,20014.489470,-52.423280,1.377630,0.000000
25%,503.000000,32804.904487,28.990415,1936.813257,0.000000
50%,1002.000000,45788.747100,41.317159,3977.287432,0.000000
75%,1501.000000,57787.565659,52.587040,6440.861434,0.000000
max,2000.000000,69995.685578,63.971796,13766.051239,1.000000
